# Data

In [1]:
import sys
sys.path.append('..')

from src.data import make_paper_dataset, build_authors_df, scrape_google_scholar, enrich_author_df_with_scholarly

DATA_PATH = '../data'

In [2]:
papers_df = make_paper_dataset(DATA_PATH)

In [3]:
papers_df.groupby('conf')['doi'].apply(lambda g: g.isna().mean())

conf
FOCS    0.000000
ICML    0.331555
NIPS    1.000000
STOC    0.000000
Name: doi, dtype: float64

In [4]:
authors_df = build_authors_df(papers_df)

/Users/shlomi/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [ ]:
authors_with_co_df = authors_df[~authors_df['only_sing']]
scholarly_results = scrape_google_scholar(authors_with_co_df, with_tor_proxy=True)
authors_df = enrich_author_df_with_scholarly(scholarly_results, authors_df)
authors_df.to_json(DATA_PATH + '/authors_FOCS_ICML_NIPS_STOC_2000_2009.json')  # TODO: Refactor